PSD + BAND POWER (ABS & RELATIVE)

In [2]:
import mne
import numpy as np
import pandas as pd
import os
from mne.time_frequency import psd_array_welch

# -------------------------
# 1. Setup Paths & Parameters
# -------------------------
BASE_DIR = r"rEEG"
SUBJ = "sub-002"
SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
DATA_DIR = os.path.join(SUBJ_DIR, "data")
EPO_FILE = os.path.join(SUBJ_DIR, "epo_002_raw.fif")

os.makedirs(DATA_DIR, exist_ok=True)

BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "high_beta_low_gamma": (30, 45)
}

# -------------------------
# 2. Load Data
# -------------------------
epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info["sfreq"]
ch_names = epochs.ch_names
data = epochs.get_data()

# -------------------------
# 2b. Reject Extreme Epochs
# -------------------------
ptp = np.ptp(data, axis=-1)
bad_epochs = np.any(ptp > 200e-6, axis=1)
data = data[~bad_epochs]

# -------------------------
# 3. Compute PSD (Welch)
# -------------------------
n_per_seg = min(512, data.shape[-1])
n_fft = n_per_seg

psds, freqs = psd_array_welch(
    data,
    sfreq=sfreq,
    fmin=1,
    fmax=45,
    n_fft=n_fft,
    n_per_seg=n_per_seg,
    average="mean",
    window="hann"
)

df = freqs[1] - freqs[0]
total_power = psds.sum(axis=-1) * df

# -------------------------
# 4. Extract Band Powers
# -------------------------
band_results = {}

for band, (fmin, fmax) in BANDS.items():
    idx = (freqs >= fmin) & (freqs <= fmax)

    abs_power = psds[:, :, idx].sum(axis=-1) * df
    abs_power_log = np.log10(abs_power + 1e-20)
    rel_power = abs_power / total_power

    band_results[band] = {
        "abs": abs_power_log,
        "rel": rel_power
    }

# -------------------------
# 5. Save PSD Arrays (INSIDE data/)
# -------------------------
np.save(os.path.join(DATA_DIR, f"{SUBJ}_psd.npy"), psds)
np.save(os.path.join(DATA_DIR, "freqs.npy"), freqs)

# -------------------------
# 6. Epoch-Level Band Power CSV (INSIDE data/)
# -------------------------
rows = []
n_epochs = data.shape[0]

for e in range(n_epochs):
    for c, ch in enumerate(ch_names):
        entry = {"epoch": e, "channel": ch}
        for band in BANDS.keys():
            entry[f"{band}_abs"] = band_results[band]["abs"][e, c]
            entry[f"{band}_rel"] = band_results[band]["rel"][e, c]
        rows.append(entry)

df_epochs = pd.DataFrame(rows)
df_epochs.to_csv(
    os.path.join(DATA_DIR, f"{SUBJ}_bandpowers_epoch.csv"),
    index=False
)


Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available


C:\Users\User\AppData\Local\Temp\ipykernel_14460\364071912.py:29: RuntimeWarning: This filename (rEEG\sub-002\epo_002_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


Not setting metadata
325 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.004 (s)


DWT coefficients / wavelet band power

In [1]:
import mne
import numpy as np
import pandas as pd
import os
import pywt


# --- 1. Setup ---
BASE_DIR = r"rEEG"
SUBJ = "sub-001"
EPO_FILE = os.path.join(BASE_DIR, SUBJ, "epo_001_raw.fif")

# Load epochs
epochs = mne.read_epochs(EPO_FILE, preload=True)
data = epochs.get_data()  # Shape: (epochs, channels, samples)
ch_names = epochs.ch_names

# --- 2. DWT Configuration ---
WAVELET = 'db4'
LEVEL = 6  # Decomposing to level 6 for 250Hz data

# Mapping of Detail Coefficients (D) to Physiological Bands (approx for fs=250)
# D3: 15.6 - 31.2 Hz (Beta)
# D4: 7.8 - 15.6 Hz  (Alpha)
# D5: 3.9 - 7.8 Hz   (Theta)
# D6: 1.9 - 3.9 Hz   (Delta)
BAND_MAP = {
    'dwt_beta': 3,
    'dwt_alpha': 4,
    'dwt_theta': 5,
    'dwt_delta': 6
}

# --- 3. Extract Wavelet Energy ---
dwt_features = []

for e_idx in range(data.shape[0]):
    for c_idx in range(data.shape[1]):
        signal = data[e_idx, c_idx, :]
        
        # Multilevel decomposition
        coeffs = pywt.wavedec(signal, WAVELET, level=LEVEL)
        
        # Initial entry for this epoch/channel
        entry = {
            'subject': SUBJ,
            'epoch': e_idx,
            'channel': ch_names[c_idx]
        }
        
        # Calculate Energy for each target band
        # Energy = sum(coefficients^2)
        for band_name, coeff_idx in BAND_MAP.items():
            # In wavedec, coeffs[0] is Approx, coeffs[1] is D6, coeffs[2] is D5...
            # The index is (LEVEL - coeff_idx + 1)
            target_coeffs = coeffs[LEVEL - coeff_idx + 1]
            energy = np.sum(np.square(target_coeffs))
            entry[band_name] = energy
            
        dwt_features.append(entry)

# --- 4. Save Results ---
df_dwt = pd.DataFrame(dwt_features)

# Save as CSV for integration with your PSD features
df_dwt.to_csv(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_features_dwt.csv"), index=False)

# Optional: Save raw coefficients as .npz if you need them for reconstruction later
# np.savez_compressed(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_dwt_coeffs.npz"), data=dwt_features)

print(f"DWT feature extraction complete for {SUBJ}. Results saved to CSV.")

Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\User\AppData\Local\Temp\ipykernel_1208\1513190370.py:14: RuntimeWarning: This filename (rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


254 matching events found
No baseline correction applied
0 projection items activated
DWT feature extraction complete for sub-001. Results saved to CSV.


Spectral Entropy

In [ ]:
import numpy as np
import pandas as pd
import os

# --- 1. Setup ---
BASE_DIR = r"rEEG"
SUBJ = "sub-001"

# Load the PSD and Frequencies we saved in the FFT step
psds = np.load(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_psd.npy")) # (epochs, channels, freqs)
freqs = np.load(os.path.join(BASE_DIR, SUBJ, "freqs.npy"))

# --- 2. Compute Spectral Entropy ---
def calculate_spectral_entropy(psd_array, freq_array, fmin=1, fmax=45):
    """
    Computes Shannon Entropy of the Power Spectral Density.
    """
    # Filter for the 1-45 Hz range specified in your notes
    idx = np.logical_and(freq_array >= fmin, freq_array <= fmax)
    psd_band = psd_array[:, :, idx]
    
    # Normalize PSD so it sums to 1 (treating it like a probability distribution)
    psd_norm = psd_band / psd_band.sum(axis=-1, keepdims=True)
    
    # Shannon Entropy formula: H = -sum(p * log2(p))
    # We add a tiny epsilon (1e-12) to avoid log(0)
    se = -np.sum(psd_norm * np.log2(psd_norm + 1e-12), axis=-1)
    
    # Normalize by log2 of number of frequency bins to bound between 0 and 1
    num_bins = psd_norm.shape[-1]
    se_normalized = se / np.log2(num_bins)
    
    return se_normalized

# Execute calculation
se_values = calculate_spectral_entropy(psds, freqs)

# --- 3. Save Results ---
# We will create a simple table for entropy to merge later
entropy_data = []
for e_idx in range(se_values.shape[0]):
    for c_idx in range(se_values.shape[1]):
        entropy_data.append({
            'epoch': e_idx,
            'channel_idx': c_idx,
            'spectral_entropy': se_values[e_idx, c_idx]
        })

df_entropy = pd.DataFrame(entropy_data)
df_entropy.to_csv(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_features_entropy.csv"), index=False)

print(f"Spectral Entropy extraction complete for {SUBJ}.")

Connectivity

In [4]:
import mne
import numpy as np
import pandas as pd
import os
from mne_connectivity import spectral_connectivity_epochs

# --- 1. Setup ---
BASE_DIR = r"rEEG"
SUBJ = "sub-001"
EPO_FILE = os.path.join(BASE_DIR, SUBJ, "epo_001_raw.fif")

# Load epochs
epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info['sfreq']

# Define bands for connectivity (standard canonical bands)
con_bands = {
    'delta': (1, 4), 
    'theta': (4, 8), 
    'alpha': (8, 12), 
    'beta': (12, 30)
}

# --- 2. Compute Connectivity (PLI) ---
# We compute this per band to see circuit-specific communication
connectivity_results = {}

for band, (fmin, fmax) in con_bands.items():
    con = spectral_connectivity_epochs(
        epochs,
        method='pli',
        mode='multitaper',
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        faverage=True, # Average across the frequencies in the band
        block_size=1000,
        n_jobs=1 # Set to higher if you have multiple cores
    )
    
    # con.get_data() returns (n_connections, 1) because faverage=True
    # We reshape it back to a (n_channels, n_channels) matrix
    con_matrix = con.get_data(output='dense')[:, :, 0]
    connectivity_results[band] = con_matrix

# --- 3. Save Results ---
# Connectivity matrices are large, so we save them as a pickle or .npy
import pickle
with open(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_connectivity_pli.pkl"), 'wb') as f:
    pickle.dump(connectivity_results, f)

# Also save a simplified CSV of "Global Connectivity" (mean PLI per band)
global_con = {f"{b}_mean_pli": [np.mean(mat)] for b, mat in connectivity_results.items()}
pd.DataFrame(global_con).to_csv(os.path.join(BASE_DIR, SUBJ, f"{SUBJ}_global_connectivity.csv"), index=False)

print(f"Connectivity extraction (PLI) complete for {SUBJ}.")

Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
254 matching events found
No baseline correction applied
0 projection items activated


C:\Users\User\AppData\Local\Temp\ipykernel_26744\789457235.py:13: RuntimeWarning: This filename (rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(EPO_FILE, preload=True)


Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1770 connections
    using t=0.000s..2.000s for estimation (501 points)
    frequencies: 1.5Hz..4.0Hz (6 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing cross-spectral density for epoch 1
    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11


C:\Users\User\AppData\Local\Temp\ipykernel_26744\789457235.py:29: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 254 events (all good), 0 – 2 s (baseline off), ~58.3 MiB, data loaded,
 '1': 254>, so metadata was not modified.
  con = spectral_connectivity_epochs(
C:\Users\User\AppData\Local\Temp\ipykernel_26744\789457235.py:29: RuntimeWarning: fmin=1.000 Hz corresponds to 2.004 < 5 cycles based on the epoch length 2.004 sec, need at least 5.000 sec epochs or fmin=2.495. Spectrum estimate will be unreliable.
  con = spectral_connectivity_epochs(


    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    computing cross-spectral density for epoch 22
    computing cross-spectral density for epoch 23
    computing cross-spectral density for epoch 24
    computing cross-spectral density for epoch 25
    computing cross-spectral density for epoch 26
    computing cross-spectral density for epoch 27
    computing cross-spectral density for epoch 28
    computing cross-spectral density for epoch 29
    computing cross-spectral density for epoch 30
    computing cross-spectral density for epoch 31


...

    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    computing cross-spectral density for epoch 22
    computing cross-spectral density for epoch 23
    computing cross-spectral density for epoch 24
    computing cross-spectral density for epoch 25
    computing cross-spectral density for epoch 26
    computing cross-spectral density for epoch 27
    computing cross-spectral density for epoch 28
    computing cross-spectral density for epoch 29
    computing cross-spectral density for epoch 30
    computing cross-spectral density for epoch 31
    computing cross-spectral density for epoch 32
    computing cross-spectral density for epoch 33
    computing cross-spectral density for epoch 34
    computing cross-spectral density for epoch 35
    computing cross-spectral density for epoch 36


C:\Users\User\AppData\Local\Temp\ipykernel_26744\789457235.py:29: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 254 events (all good), 0 – 2 s (baseline off), ~58.3 MiB, data loaded,
 '1': 254>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    computing cross-spectral density for epoch 22
    computing cross-spectral density for epoch 23
    computing cross-spectral density for epoch 24
    computing cross-spectral density for epoch 25
    computing cross-spectral density for epoch 26
    computing cross-spectral density for epoch 27
    computing cross-spectral density for epoch 28
    computing cross-spectral density for epoch 29
    computing cross-spectral density for epoch 30
    computing cross-spectral density for epoch 31
    computing cross-spectral density for epoch 32
    computing cross-spectral density for epoch 33


C:\Users\User\AppData\Local\Temp\ipykernel_26744\789457235.py:29: RuntimeWarning: There were no Annotations stored in <EpochsFIF | 254 events (all good), 0 – 2 s (baseline off), ~58.3 MiB, data loaded,
 '1': 254>, so metadata was not modified.
  con = spectral_connectivity_epochs(


    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    computing cross-spectral density for epoch 22
    computing cross-spectral density for epoch 23
    computing cross-spectral density for epoch 24
    computing cross-spectral density for epoch 25
    computing cross-spectral density for epoch 26
   